# HW2, COMS 4995_005, Deep Learning

## Team Information

Team Member1 (Name,UNI): Richard Kennedy, rk2860

Python Version: 2
Utilizing Pytorch

In [4]:
import torch
import cPickle
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchtext.vocab as Vocab


ship


## Part 1

In [ ]:


class LSTM(nn.Module):
     def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(LSTM, self).__init__()
        
        
        self.Wix = nn.Linear(embded_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)
        self.Wfx = nn.Linear(embed_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)
        self.Wox = nn.Linear(embed_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)
        self.Wcx = nn.Linear(embed_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)
        
        res50 = torchvision.models.resnet50(pretrained=True)
        self.res50NoGrad = torch.nn.Sequential(*list(res50.children())[:-2])
        #Every layer besides the last layer does not need to be trained
        for param in self.res50NoGrad.parameters():
            param.requires_grad = False
        
        #set embed_size to dimensions of self.res50Last
        #output should be second to last layer
        self.res50Last = torch.nn.Sequential(*list(res50.children())[-1])
        
        
        self.glove = Vocab.GloVe(name='840B', dim= embed_size)
        
        
        
        

    def forward(self, x, m = None, c = None, CNN = False ):
        
        if not CNN:
            #glove requires x to be one hot word encoding?
            x = self.glove(x)
            i = F.sigmoid(self.Wix(x)+self.Wim(m))
            f = F.sigmoid(self.Wfx(x)+self.Wfm(m))
            o = F.sigmoid(self.Wox(x)+self.Wom(m))
            h = F.tanh(Wcx(x) + Wcm(m))
            newC = f*c + i*h
            newM = o*newC
            #prediction dimension should line up with vocab_size
            #fine tune hidden dimension to make that work
            
            #log_softmax might not be necessary -> due to backprop
            prediction = F.log_softmax(newM,)
            
            return prediction, newC, newM
        
        else:
            x= self.res50NoGrad(x)
            x = self.res50Last(x)
            i = F.sigmoid(self.Wix(x))
            f = F.sigmoid(self.Wfx(x))
            o = F.sigmoid(self.Wox(x))
            h = F.tanh(Wcx(x))
            c = f*c + i*h
            m = o*c
            
            return None, c, m

/Users/Richard/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:75: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Part 2

## Extra Credit